In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, Conv2D, Activation, Flatten, Dense, Input, Lambda, Permute, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow import sparse

import matplotlib.pyplot as plt
from tqdm import tqdm, trange

from dataprep.event import Event
import numpy as np

In [2]:
# Load data
import pickle, gzip

class MyUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "Event" and module == 'event':
            return Event
        else:
            return pickle.Unpickler.find_class(self, module, name)

# with open('data/ES_clean_0.pkl', 'rb') as f:
def gen_event(quiet=False):
    for file_id in range(50):
        if not quiet:
            print(f"Starting File {file_id}")
        dataset = MyUnpickler(open(f'data/ES_clean_{file_id}.pkl', 'rb')).load()
        for event in dataset:
            yield event

nevents = sum(1 for _ in gen_event(quiet=True))

2022-05-03 23:27:11.381052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 23:27:11.412152: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 23:27:11.412308: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-03 23:27:11.412663: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
learning_rate = 1e-3
map_shape = (12, 2560, 4492)
x = Input(shape=map_shape)
# y = Lambda(sparse.to_dense)(x)
# y = Reshape(map_shape)(y)
y = Permute((2, 3, 1), input_shape=map_shape, name='swap_axis')(x)

y = Conv1D(filters=16, kernel_size=128, activation='relu', name="conv_1")(y)
y = Conv2D(filters=16, kernel_size=(4, 16), strides=(4, 16), activation='relu', name="conv_2")(y)
y = Flatten(name='flatten')(y)
y = Dense(1, activation='relu', name='output')(y)
model = Model(x, y, name='model')
model.compile(loss='mean_absolute_error', optimizer=Adam(learning_rate))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 2560, 4492)]  0         
                                                                 
 swap_axis (Permute)         (None, 2560, 4492, 12)    0         
                                                                 
 conv_1 (Conv1D)             (None, 2560, 4365, 16)    24592     
                                                                 
 conv_2 (Conv2D)             (None, 640, 272, 16)      16400     
                                                                 
 flatten (Flatten)           (None, 2785280)           0         
                                                                 
 output (Dense)              (None, 1)                 2785281   
                                                                 
Total params: 2,826,273
Trainable params: 2,826,273
Non-train

In [ ]:
epochs = 5
loss_history = []
for idx in range(epochs):
    all_loss_in_epoch = []
    pbar = tqdm(gen_event(quiet=True), total=nevents)
    for event in pbar:
        X_train = event.digit_map
        Y_train = event.e_energy
        X_train = tf.cast(X_train, tf.int16)
        X_dense = tf.expand_dims(sparse.to_dense(X_train), axis=0)
        y = tf.constant([Y_train])
        loss = model.train_on_batch([X_dense], [y])
        all_loss_in_epoch.append(loss)
        pbar.set_postfix({'loss': loss})
    print(np.mean(all_loss_in_epoch))
    loss_history.append(np.mean(all_loss_in_epoch))


100%|████████████████████████████████████████████████████████████████████| 1000/1000 [15:05<00:00,  1.10it/s, loss=3.83]


11.95494685435295


100%|████████████████████████████████████████████████████████████████████| 1000/1000 [15:07<00:00,  1.10it/s, loss=3.83]


11.955596037626266


 13%|████████▋                                                            | 126/1000 [01:55<13:23,  1.09it/s, loss=48.2]


In [ ]:
plt.semilogy(loss_history)
# plt.ylim(top=1e4)